# Review Radar - FastAPI Backend 

## 1. Imports and Dependencies

In [8]:
# Core FastAPI and web framework imports
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import uvicorn

# Machine Learning and NLP imports
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Utility imports
import logging
import asyncio
import json
from datetime import datetime

print(" All imports successful!")
print(f" PyTorch CUDA available: {torch.cuda.is_available()}")
print(f" PyTorch version: {torch.__version__}")

 All imports successful!
 PyTorch CUDA available: True
 PyTorch version: 2.8.0+cu126


## 2. Configure Logging

In [9]:
# Configure logging for better debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Test logging
logger.info(" Logging configured successfully!")
print("Logger ready for use")

2025-09-27 18:39:05,060 - __main__ - INFO -  Logging configured successfully!


Logger ready for use


## 3. Data Models (Pydantic)

In [10]:
# Pydantic models for request/response validation

class ReviewAnalysisRequest(BaseModel):
    """Request model for sentiment analysis"""
    reviews: List[str] = Field(
        ..., 
        description="List of review texts to analyze", 
        min_items=1,
        example=["This product is amazing!", "Terrible quality, very disappointed"]
    )

class ReviewSentiment(BaseModel):
    """Individual review sentiment result"""
    review: str
    sentiment: str
    confidence: float

class AnalysisResponse(BaseModel):
    """Complete analysis response with summary"""
    results: List[ReviewSentiment]
    summary: Dict[str, int]
    total_reviews: int
    processing_time_ms: float = 0.0

# Test the models
test_request = ReviewAnalysisRequest(reviews=["Great product!", "Not good at all"])
print(" Pydantic models created successfully!")
print(f" Test request: {test_request.model_dump()}")

 Pydantic models created successfully!
 Test request: {'reviews': ['Great product!', 'Not good at all']}


## 4. Initialize Sentiment Analysis Model

In [11]:
# Global sentiment analyzer (will be initialized once)
sentiment_analyzer = None

def initialize_sentiment_model():
    """
    Initialize the sentiment analysis model on startup
    Using DistilBERT fine-tuned on Stanford Sentiment Treebank (SST-2)
    """
    global sentiment_analyzer
    
    try:
        logger.info("🤖 Loading sentiment analysis model...")
        
        # Load the specific model requested
        model_name = "distilbert-base-uncased-finetuned-sst-2-english"
        
        # Initialize the pipeline with the model
        sentiment_analyzer = pipeline(
            "sentiment-analysis",
            model=model_name,
            tokenizer=model_name,
            return_all_scores=True,  # Get confidence scores
            device=0 if torch.cuda.is_available() else -1  # Use GPU if available
        )
        
        device_info = 'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'
        logger.info(f" Model loaded successfully! Using device: {device_info}")
        
        # Test the model with a sample text
        test_result = sentiment_analyzer("This is a great product!")
        logger.info(f" Model test successful: {test_result}")
        
        return True
        
    except Exception as e:
        logger.error(f" Failed to load sentiment model: {str(e)}")
        return False

# Initialize the model
success = initialize_sentiment_model()
print(f" Model initialization: {'SUCCESS' if success else 'FAILED'}")

2025-09-27 18:39:05,083 - __main__ - INFO - 🤖 Loading sentiment analysis model...
Device set to use cuda:0
Device set to use cuda:0
2025-09-27 18:39:06,314 - __main__ - INFO -  Model loaded successfully! Using device: GPU (CUDA)
2025-09-27 18:39:06,314 - __main__ - INFO -  Model loaded successfully! Using device: GPU (CUDA)
2025-09-27 18:39:06,355 - __main__ - INFO -  Model test successful: [[{'label': 'NEGATIVE', 'score': 0.00012437114492058754}, {'label': 'POSITIVE', 'score': 0.9998756647109985}]]
2025-09-27 18:39:06,355 - __main__ - INFO -  Model test successful: [[{'label': 'NEGATIVE', 'score': 0.00012437114492058754}, {'label': 'POSITIVE', 'score': 0.9998756647109985}]]


 Model initialization: SUCCESS


## 5. Helper Functions

In [12]:
def map_sentiment_label(label: str) -> str:
    """
    Map model output labels to standardized sentiment labels
    DistilBERT SST-2 outputs: POSITIVE, NEGATIVE
    We map to: Positive, Negative, Neutral (though SST-2 doesn't have neutral)
    """
    label_mapping = {
        "POSITIVE": "Positive",
        "NEGATIVE": "Negative",
        "NEUTRAL": "Neutral"  # For future models that support neutral
    }
    return label_mapping.get(label.upper(), label)

def analyze_reviews_batch(reviews: List[str]) -> List[ReviewSentiment]:
    """
    Analyze sentiment for a batch of reviews
    Returns structured results with confidence scores
    """
    if not sentiment_analyzer:
        raise Exception("Sentiment model not initialized")
    
    try:
        # Process all reviews in batch for efficiency
        logger.info(f"🔍 Analyzing {len(reviews)} reviews...")
        start_time = datetime.now()
        
        # Get predictions for all reviews
        predictions = sentiment_analyzer(reviews)
        
        results = []
        for i, (review, prediction) in enumerate(zip(reviews, predictions)):
            # Get the highest confidence prediction
            best_prediction = max(prediction, key=lambda x: x['score'])
            
            sentiment = map_sentiment_label(best_prediction['label'])
            confidence = round(best_prediction['score'], 4)
            
            results.append(ReviewSentiment(
                review=review,
                sentiment=sentiment,
                confidence=confidence
            ))
            
            logger.debug(f" Review {i+1}: {sentiment} (confidence: {confidence})")
        
        processing_time = (datetime.now() - start_time).total_seconds() * 1000
        logger.info(f" Processing completed in {processing_time:.2f}ms")
        
        return results
        
    except Exception as e:
        logger.error(f" Error during sentiment analysis: {str(e)}")
        raise

def calculate_summary(results: List[ReviewSentiment]) -> Dict[str, int]:
    """
    Calculate aggregate counts of sentiment labels
    """
    summary = {"Positive": 0, "Negative": 0, "Neutral": 0}
    
    for result in results:
        if result.sentiment in summary:
            summary[result.sentiment] += 1
    
    return summary

print(" Helper functions defined successfully!")

 Helper functions defined successfully!


## 6. Test Individual Functions

In [15]:
# Test the sentiment analysis with sample reviews
test_reviews = [
    "This product is absolutely amazing! Best purchase ever!",
    "Terrible quality, broke after one day. Very disappointed.",
    "It's okay, nothing special but does the job.",
    "Love it! Highly recommend to everyone!",
    "Worst product I've ever bought. Complete waste of money."
]

print(" Testing sentiment analysis with sample reviews...")
print("="*60)

try:
    # Analyze the test reviews
    results = analyze_reviews_batch(test_reviews)
    
    # Display results
    for i, result in enumerate(results, 1):
        print(f"{i}. Review: {result.review[:50]}...")
        print(f"   Sentiment: {result.sentiment} (Confidence: {result.confidence})")
        print()
    
    # Calculate and display summary
    summary = calculate_summary(results)
    print(" Summary:")
    for sentiment, count in summary.items():
        print(f"   {sentiment}: {count}")
    
    print(f"\n Successfully analyzed {len(results)} reviews!")
    
except Exception as e:
    print(f" Error during testing: {str(e)}")

2025-09-27 18:39:06,892 - __main__ - INFO - 🔍 Analyzing 5 reviews...
2025-09-27 18:39:06,912 - __main__ - INFO -  Processing completed in 19.10ms
2025-09-27 18:39:06,912 - __main__ - INFO -  Processing completed in 19.10ms


 Testing sentiment analysis with sample reviews...
1. Review: This product is absolutely amazing! Best purchase ...
   Sentiment: Positive (Confidence: 0.9999)

2. Review: Terrible quality, broke after one day. Very disapp...
   Sentiment: Negative (Confidence: 0.9997)

3. Review: It's okay, nothing special but does the job....
   Sentiment: Positive (Confidence: 0.9995)

4. Review: Love it! Highly recommend to everyone!...
   Sentiment: Positive (Confidence: 0.9999)

5. Review: Worst product I've ever bought. Complete waste of ...
   Sentiment: Negative (Confidence: 0.9998)

 Summary:
   Positive: 3
   Negative: 2
   Neutral: 0

 Successfully analyzed 5 reviews!
